In [11]:
# %%
import warnings
warnings.filterwarnings('ignore')

# Basic libraries
import random
import time
import numpy  as np
import pandas as pd
import os
# Sklearn library
from sklearn.preprocessing   import StandardScaler, RobustScaler
# User libraries
from utils.data_loading import Synthetic_dataset, TWINS_dataset, IHDP_dataset, ACIC_dataset
from utils.metrics      import PEHE, ATE
print('[INFO] All libraries were imported')



# Random generators initialization
seed=42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
print('[INFO] Random generators were initialized')

[INFO] All libraries were imported
[INFO] Random generators were initialized


In [12]:
problem = "TWINS" # {"IHDP", "Synthetic", "TWINS", "ACIC"}
path = "Data/TWINS/" # {"Data/Synthetic/", "Data/IHDP/", "Data/TWINS/", "Data/ACIC/"}
filename = f"./Results/{problem}_C-XGBoost.csv"


if "Synthetic" in problem:
    DataLoader = Synthetic_dataset(path=path)
elif "IHDP" in problem:
    DataLoader = IHDP_dataset(path=path)
elif "TWINS" in problem:
    DataLoader = TWINS_dataset(path=path)
elif "ACIC" in problem:
    DataLoader = ACIC_dataset(path=path, train_size=0.8, random_state=1983)  

In [13]:
results = {'ATE': [], 'Error_ATE': [], 'Error_PEHE':[]} 
for idx in range(DataLoader.nProblems):

    # Start timer
    #
    start1 = time.time()
    
    
    # Load training data
    trainX, trainT, trainY, train_potential_Y = DataLoader.getTraining( idx )

    # Load testing data
    testX, testT, testY, test_potential_Y  = DataLoader.getTesting( idx )
    #
    print('Simulation: ', idx)
    print('[INFO] Dataset imported')
    print('[INFO] Number of training instances: ', trainX.shape[0])
    
 
    # Setup scaler for inputs
    #
    scalerX = RobustScaler()
    trainX  = scalerX.fit_transform( trainX )
    testX   = scalerX.transform( testX )


    tt = np.array([[1,0] if x == 0 else [0,1] for x in trainT]).flatten()

    def custom_loss(y_true:np.ndarray=None, y_pred:np.ndarray=None)->(np.ndarray,np.ndarray):
        grad = 2*(y_pred.flatten() - y_true.flatten()) * tt
        hess = (0*y_pred.flatten() + 2)  * tt

        return grad, hess
    


    # Setup XGBoost
    #
    # import xgboost
    # model = xgboost.XGBRegressor(n_estimators=200, 
    #                              max_depth=4, 
    #                              objective=custom_loss, 
    #                              learning_rate=1e-2, 
    #                             #  booster="gblinear",
    #                              n_jobs=-1,
    #                              tree_method="hist", 
    #                              multi_strategy="multi_output_tree")
    import xgboost
    model = xgboost.XGBRegressor(n_estimators=200, 
                                 max_depth=5, 
                                 objective=custom_loss, 
                                 learning_rate=5e-5, 
                                #  booster="gblinear",
                                 #
                                 reg_alpha = 0,
                                 reg_lambda = 1.0,
                                 gamma = 1.0,
                                 min_child_weight = 2.0,
                                 max_leaves = 2,
                                 #
                                 n_jobs=-1,
                                 tree_method="hist", 
                                 multi_strategy="multi_output_tree")
    
    # Create outputs for DragonNet (concatenate Y & T)
    #
    if "ACIC" in problem:
        scalerY       = RobustScaler()
        trainY_scaled = scalerY.fit_transform( trainY.reshape(-1,1) ).squeeze(-1)      
        yt_train = np.concatenate([trainY_scaled.reshape(-1,1), trainT.reshape(-1,1)], axis = 1)
    else:
        yt_train = np.concatenate([trainY.reshape(-1,1), trainT.reshape(-1,1)], axis = 1)
    

    # Train model
    #
    # model.fit(trainX, yt_train);
    model.fit(trainX, yt_train, eval_set=[(trainX, train_potential_Y), (testX, test_potential_Y)], verbose=50);
    print('[INFO] Model trained')    
    
    
    
    
    #
    #
    # *** Predictions ***
    #
    #       

    # Get predictions
    #
    test_y_hat = model.predict( testX )

    if "ACIC" in problem:
        test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) ).squeeze()
        test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) ).squeeze()    
    
    # ATE
    #
    real_ATE = ( test_potential_Y[:,1] - test_potential_Y[:,0] ).mean()
    
    
    # Error PEHE
    #
    Error_PEHE = PEHE(test_potential_Y, test_y_hat)
    
    
    # Error ATE
    #
    Error_ATE = ATE(test_potential_Y, test_y_hat)  
    
        
    # Store errors of PEHE and ATE
    #
    results['ATE']            += [ np.round(real_ATE,   6) ]
    results['Error_ATE']      += [ np.round(Error_ATE,  6) ]
    results['Error_PEHE']     += [ np.round(Error_PEHE, 6) ]
    print('[INFO] Time %.2f seconds\n\n' % (time.time() - start1))

    # from sklearn import metrics
    # print('0: ', metrics.mean_absolute_error(test_potential_Y[:,0], test_y_hat[:,0]))
    # print('1: ', metrics.mean_absolute_error(test_potential_Y[:,1], test_y_hat[:,1]))    


    
    # Save results (at each iteration)
    df = pd.DataFrame( results )
    df['Problem'] = [f"{problem} {x}" for x in df.index]
    df.to_csv(filename, index=False)

    # print(results['Error_ATE'])
    # print(results['Error_PEHE'])
    # print(50*"-" + "\n")

Simulation:  0
[INFO] Dataset imported
[INFO] Number of training instances:  19124
[0]	validation_0-rmse:0.50000	validation_1-rmse:0.50000
[50]	validation_0-rmse:0.50000	validation_1-rmse:0.50000
[99]	validation_0-rmse:0.50000	validation_1-rmse:0.50000
[INFO] Model trained
[INFO] Time 5.74 seconds


Simulation:  1
[INFO] Dataset imported
[INFO] Number of training instances:  19124
[0]	validation_0-rmse:0.50000	validation_1-rmse:0.50000
[50]	validation_0-rmse:0.50000	validation_1-rmse:0.50000


KeyboardInterrupt: 

In [ ]:
np.where(trainT==0)[0].size, np.where(trainT==1)[0].size

(2157, 16967)